In [1]:
import torch
import os
import yaml
import pickle
from tqdm import tqdm
import numpy as np
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
from pathlib import Path
from collections import defaultdict
from matplotlib.ticker import FuncFormatter
import pandas as pd
from torch.utils.data import DataLoader
import torchvision.transforms as tt

from tiatoolbox.models.engine.nucleus_instance_segmentor import NucleusInstanceSegmentor
from utils import PGA
from utils.utils_nucleus import *
from utils.PGA import PGA
import warnings

warnings.filterwarnings("ignore")
print("cuda available? ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vis_scale = 0.005
enlarge = 10
perc_wpx = 0.85
perc_bpx = 0.05
real_enlarge = int(enlarge / vis_scale)


def log_tick_formatter(x, pos):
    return f"{np.expm1(x):.0f}"

|2026-02-08|12:36:40.636| [WARNING] /usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

|2026-02-08|12:36:44.538| [WARNING] /usr/local/lib/python3.12/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()



cuda available?  True


In [2]:
hemato_1 = (73, 119, 185.0)
eosin = (245, 143.0, 204)
safran = (248.0, 245, 152)
hemato_2 = (123, 153, 198)
Wgt = -np.log(np.array([hemato_1, eosin, safran, hemato_2]).T / 255)
pga = PGA(Wgt, device=device)
Lambda = 1.15e-1
poids = (1.5, 1)
kernel_size = 7
clean_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))

In [3]:
slide_name = "36H"
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

tumor_checkpoints = "../" + config["paths"]["pth_to_tumor_ckpts"]
patches_bis_path = "../" + config["paths"]["pth_to_patches"].replace(
    "patches", "patches_bis"
)
print(patches_bis_path)
chkpt_tumor = torch.load(f"{tumor_checkpoints}/{slide_name}_preds_probas_checkpoint.pt")

../data/patches_bis/


In [4]:
chkpt_tumor.keys()

dict_keys(['scaled_slide', 'coords_x', 'coords_y', 'har_mean_proba', 'har_mean_preds', 'geo_mean_proba', 'geo_mean_preds', 'arith_mean_proba', 'arith_mean_preds'])

In [5]:
y_har_preds = chkpt_tumor["har_mean_preds"]
coords_x, coords_y = chkpt_tumor["coords_x"], chkpt_tumor["coords_x"]

In [10]:
coords_x, coords_y = [], []
for patch in os.listdir(f"{patches_bis_path}/{slide_name}"):
    _, _, x, _, y = patch[:-4].split("_")
    coords_x.append(int(x))
    coords_y.append(int(y))

In [6]:
all_x, all_y, median_areas, mean_areas, nucleocytos, densities, anisos, median_vars = (
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
)

y_harmonic = y_har_preds.numpy()
for j in tqdm(range(len(y_har_preds))):
    x, y, p = coords_x[j], coords_y[j], y_harmonic[j]
    if p != 0:
        try:
            patch = f"patch_x_{x}_y_{y}.jpg"
            im = plt.imread(f"{patches_bis_path}{slide_name}/{patch}")
            V = vectorize(im)
            N, M, _ = im.shape
            H0 = np.maximum((pinv(Wgt) @ V), 0)
            im_He = getHstain(V, Wgt, H0, Lambda, pga, poids, n=N)
            mask = getNucleusMask(im_He, gaussian_filter=(7, 7))
            clean_mask = getCleanMask(mask, clean_kernel)
            contour_im0, contours = detectContours(im, clean_mask)
            image, filtred_contours = detectNucleus(
                contour_im0, contours, inf_p=45, inf_a=45
            )
            final_im = segmentNucleus(im, filtred_contours)
            (
                density,
                mean_area,
                median_area,
                anisocaryose,
                median_variance_area,
                nucleocyto_idx,
            ) = computeFeatures(filtred_contours, final_im)
            all_x.append(x)
            all_y.append(y)
            densities.append(density)
            mean_areas.append(mean_area)
            median_areas.append(median_area)
            anisos.append(anisocaryose)
            nucleocytos.append(nucleocyto_idx)
            median_vars.append(median_variance_area)
        except Exception as e:
            print(f"Error in {x} {y}: {e}")
            continue

 51%|█████     | 1757/3459 [00:00<00:00, 17563.57it/s]

Error in 176608 176608: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_176608_y_176608.jpg'
Error in 239968 239968: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_239968_y_239968.jpg'
Error in 230752 230752: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_230752_y_230752.jpg'
Error in 178912 178912: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_178912_y_178912.jpg'
Error in 215776 215776: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_215776_y_215776.jpg'
Error in 215776 215776: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict

100%|██████████| 3459/3459 [00:00<00:00, 16963.52it/s]

Error in 210016 210016: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_210016_y_210016.jpg'
Error in 185824 185824: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_185824_y_185824.jpg'
Error in 193888 193888: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_193888_y_193888.jpg'
Error in 197344 197344: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_197344_y_197344.jpg'
Error in 227296 227296: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict-Early-Rec-of-HCC/data/patches_bis/36H/patch_x_227296_y_227296.jpg'
Error in 183520 183520: [Errno 2] No such file or directory: '/workspace/ML-Model-Integrating-CPath-to-Predict